In [1]:
import os
import sys
parent_dir = os.path.abspath('.')
sys.path.append(parent_dir + '/bilinear_interp_tim')
sys.path.append(parent_dir + '/dictionary_learning')
sys.path.append(parent_dir)
import torch as t
from einops import rearrange, repeat, reduce,einsum
from language import Transformer, Sight


/home/tim/miniconda3/envs/new/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Transformer.from_pretrained("tdooms/fw-nano", device="cuda")
model = Sight(model)

In [3]:
model._model.config.dataset = 'fineweb'
mlp_0 = model._model.transformer.h[0].mlp
w_l_0 = model._model.transformer.h[0].mlp.w.weight.chunk(2, dim=0)[0]
w_r_0 = model._model.transformer.h[0].mlp.w.weight.chunk(2, dim=0)[1]
w_p_0 = model._model.transformer.h[0].mlp.p.weight
l_bias, r_bias = model._model.transformer.h[0].mlp.w.bias.chunk(2, dim = 0)
w_p_bias = model._model.transformer.h[0].mlp.p.bias

In [5]:
t.manual_seed(123121)
rand_vec_1 = t.randn(1024, device = 'cuda')/10
rand_vec_2 = t.randn(1024, device = 'cuda')/10

test_input = rand_vec_1 + rand_vec_2
output = mlp_0(test_input)
print(output.norm())
normed_output = output / output.norm()

tensor(2.5353, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)


In [6]:
interaction = t.einsum("mi,mj,om,...o->...ij", w_l_0, w_r_0, w_p_0, normed_output)
interaction = 0.5 * (interaction + interaction.mT)
interaction_output = test_input.unsqueeze(0) @ interaction @ test_input.unsqueeze(1)
print(interaction_output)

tensor([[0.5113]], device='cuda:0', grad_fn=<MmBackward0>)


In [7]:
linear_component = t.diag(l_bias) @ w_l_0 +  t.diag(r_bias) @ w_r_0
linear_hidden = linear_component @ test_input.unsqueeze(1)
linear_output = normed_output.unsqueeze(0) @ w_p_0 @ linear_hidden
print(linear_output)

tensor([[0.0012]], device='cuda:0', grad_fn=<MmBackward0>)


In [8]:
hidden_bias = l_bias*r_bias
bias_out = normed_output.unsqueeze(0) @ w_p_0 @ hidden_bias.unsqueeze(1)
print(bias_out)

tensor([[-0.0092]], device='cuda:0', grad_fn=<MmBackward0>)


In [9]:
mlp_out_bias_out = t.dot(normed_output, w_p_bias)
print(mlp_out_bias_out)

tensor(1.7390, device='cuda:0', grad_fn=<DotBackward0>)


In [10]:
decomposed_contributions = mlp_out_bias_out + bias_out + linear_output + interaction_output
decomposed_contributions

tensor([[2.2423]], device='cuda:0', grad_fn=<AddBackward0>)

In [11]:
output.norm()==decomposed_contributions.norm()

tensor(False, device='cuda:0')

```python
def q_linear_and_bias(self, out_dir: torch.Tensor, project_mat: Optional[torch.Tensor] = None):
    if project_mat is not None:
        assert project_mat.shape[0] == out_dir.shape[0], f"Projection matric shape {project_mat.shape} must match output direction shape {out_dir.shape}"
    model, layer = self.model, self.layer
    interaction = torch.einsum("mi,mj,om,...o->...ij", model.w_l[layer], model.w_r[layer], model.w_p[layer], out_dir)
    interaction = 0.5 * (interaction + interaction.mT)
    if project_mat is not None:
        interaction = torch.einsum("il,jk,...ij->...lk", project_mat, project_mat, interaction)
    
    l_bias, r_bias = model.transformer.h[layer].mlp.w.bias.chunk(2, dim = 0)

    linear_component = torch.diag(l_bias) @ model.w_l[layer]  +  torch.diag(r_bias) @ model.w_r[layer]

    pure_bias = l_bias*r_bias

    return interaction, linear_component, pure_bias
```